In [17]:
import pandas as pd
import numpy as np
import re
from transformers import BertModel, BertTokenizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer, util
import sklearn.metrics as metrics
from sklearn.metrics import precision_score, recall_score, accuracy_score
from tqdm import tqdm
import random

In [2]:
dataFile = pd.read_excel("student_info.xlsx",sheet_name='Sheet1')
dataFile

,ID,UserID,UserQuery,errorClasses,taskid,timestamp,E0,E1,E2,E3,...,E12,E13,E14,E15,E16,E17,E18,E19,E20,E21
0,26373,1101,select name from Customer,0,454,2020-06-15 13:16:04,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,26374,1101,select C.name from Customer as C,0,454,2020-06-15 13:16:26,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,27282,1103,select name from customer,0,454,2020-06-16 23:20:56,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28499,1104,SELECT name FROM Customer;,0,454,2020-06-18 22:58:57,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,26942,1107,Select name from customer;,0,454,2020-06-16 11:51:33,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50329,199184,2478,"CREATE TABLE PRODUCER( vineyard VARCHAR(30), ...",8,1220,2023-07-16 20:41:50,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50330,199185,2478,"CREATE TABLE PRODUCER( vineyard VARCHAR(30), ...",0,1220,2023-07-16 20:43:02,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50331,199520,2479,CREATE TABLE ERZEUGER(anbaugebiet varchar(100)...,9,1220,2023-07-25 19:10:32,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50332,199521,2479,CREATE TABLE ERZEUGER(anbaugebiet varchar(100)...,9,1220,2023-07-25 19:11:02,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
selected_rows = dataFile[(dataFile['E0'] == 1) | (dataFile['E1'] == 1) | (dataFile['E2'] == 1) | (dataFile['E3'] == 1) | (dataFile['E4'] == 1) | (dataFile['E5'] == 1) | (dataFile['E6'] == 1) | (dataFile['E7'] == 1) | (dataFile['E8'] == 1) | (dataFile['E9'] == 1) | (dataFile['E10'] == 1) | (dataFile['E11'] == 1) | (dataFile['E12'] == 1) | (dataFile['E13'] == 1) | (dataFile['E14'] == 1) | (dataFile['E15'] == 1) | (dataFile['E16'] == 1) | (dataFile['E17'] == 1) | (dataFile['E18'] == 1) | (dataFile['E19'] == 1) | (dataFile['E20'] == 1) | (dataFile['E21'] == 1)]
df = pd.DataFrame(selected_rows)
df = df.reset_index()
df

,index,ID,UserID,UserQuery,errorClasses,taskid,timestamp,E0,E1,E2,...,E12,E13,E14,E15,E16,E17,E18,E19,E20,E21
0,29,26290,1143,SELECT Name FROM Customer;,0,454,2020-06-15 12:07:01,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,44,33600,1158,"SELECT customer.Cid, customer.NameFROM Custome...",0,454,2020-06-26 08:43:20,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,70,28417,1216,select * from Customer;,2,454,2020-06-18 18:17:16,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,87,28121,1246,"select cid, name FROM customer GROUP BY name",2,454,2020-06-18 12:01:15,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,98,31056,1277,select * from customer,2,454,2020-06-20 20:42:07,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,50164,199669,2475,CREATE TABLE WINE( name VARCHAR (50) DEFAULT ...,0,1219,2023-07-26 14:43:12,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4319,50179,199527,2479,CREATE TABLE WEIN(farbe varchar(100) DEFAULT (...,0,1219,2023-07-25 19:40:44,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4320,50184,199691,2479,CREATE TABLE WEIN( farbe varchar(100) DEFAULT...,0,1219,2023-07-26 15:29:23,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4321,50211,190044,2278,create table PRODUCER( vineyard varchar(20) n...,0,1220,2023-05-04 14:05:29,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
columns_with_1 = []
for i in range(0, 4323):
    name = df.columns[df.iloc[i].eq(1)].tolist()
    columns_with_1.append(name[0])
df['Severe Error'] = columns_with_1
df

,index,ID,UserID,UserQuery,errorClasses,taskid,timestamp,E0,E1,E2,...,E13,E14,E15,E16,E17,E18,E19,E20,E21,Severe Error
0,29,26290,1143,SELECT Name FROM Customer;,0,454,2020-06-15 12:07:01,1,0,0,...,0,0,0,0,0,0,0,0,0,E0
1,44,33600,1158,"SELECT customer.Cid, customer.NameFROM Custome...",0,454,2020-06-26 08:43:20,1,0,0,...,0,0,0,0,0,0,0,0,0,E0
2,70,28417,1216,select * from Customer;,2,454,2020-06-18 18:17:16,0,0,1,...,0,0,0,0,0,0,0,0,0,E2
3,87,28121,1246,"select cid, name FROM customer GROUP BY name",2,454,2020-06-18 12:01:15,0,0,1,...,0,0,0,0,0,0,0,0,0,E2
4,98,31056,1277,select * from customer,2,454,2020-06-20 20:42:07,0,0,1,...,0,0,0,0,0,0,0,0,0,E2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,50164,199669,2475,CREATE TABLE WINE( name VARCHAR (50) DEFAULT ...,0,1219,2023-07-26 14:43:12,1,0,0,...,0,0,0,0,0,0,0,0,0,E0
4319,50179,199527,2479,CREATE TABLE WEIN(farbe varchar(100) DEFAULT (...,0,1219,2023-07-25 19:40:44,1,0,0,...,0,0,0,0,0,0,0,0,0,E0
4320,50184,199691,2479,CREATE TABLE WEIN( farbe varchar(100) DEFAULT...,0,1219,2023-07-26 15:29:23,1,0,0,...,0,0,0,0,0,0,0,0,0,E0
4321,50211,190044,2278,create table PRODUCER( vineyard varchar(20) n...,0,1220,2023-05-04 14:05:29,1,0,0,...,0,0,0,0,0,0,0,0,0,E0


In [5]:
keyword_mappings = pd.read_excel("DBC20-23.xlsx",sheet_name='Mapping')
error_mappings = pd.read_excel("DBC20-23.xlsx",sheet_name='Error Description')

In [6]:
def get_task_description(taskid):
    query = ''
    for i in range(len(keyword_mappings)):
        try:
            listing = keyword_mappings['Task ID'][i].split(',')
        except:
            listing = []
            l = keyword_mappings['Task ID'][i]
            listing.append(l)
        if(str(taskid) in listing  or taskid in listing):
            query = keyword_mappings['Task Description'][i]
            
    query = re.sub(r'\b(get|list|query|which|determine|return|retrieve|output)\b', 'select', query, flags=re.IGNORECASE)
    query = re.sub(r'\b(length|how many|number of)\b', 'count', query, flags=re.IGNORECASE)
    query = re.sub(r'\badd\b', 'insert', query, flags=re.IGNORECASE)
    query = re.sub(r'\bempty\b', 'delete', query, flags=re.IGNORECASE)
    query = re.sub(r'\brelation\b', 'table', query, flags=re.IGNORECASE)
    query = re.sub(r'\b(that|where)\b', 'where', query, flags=re.IGNORECASE)

    return(query)

In [7]:
def get_error_description(error_class):
    err_desc = ''
    for i in range(len(error_mappings)):
        if(error_class == error_mappings['Error Label'][i]):
            err_desc = error_mappings['Error Description'][i]
    return(err_desc)

In [8]:
def process_slides(text):
    start_pattern = r'^.*?\n'
    end_pattern = r'\n.*$'
    result = re.sub(start_pattern, '', text, count=1)
    result = re.sub(end_pattern, '', result)
    result = result.replace('\n', ' ')  
    return(result)

In [9]:
data = pd.read_csv('slides.csv')

data = data[data['Slide Number'] != 'PDF-2, Pg1']
data = data[data['Slide Number'] != 'PDF-2, Pg10']
data = data[data['Slide Number'] != 'PDF-2, Pg16']
data = data[data['Slide Number'] != 'PDF-2, Pg23']
data = data[data['Slide Number'] != 'PDF-2, Pg36']
data = data[data['Slide Number'] != 'PDF-2, Pg41']

data = data[data['Slide Number'] != 'PDF-6, Pg1']
data = data[data['Slide Number'] != 'PDF-6, Pg13']
data = data[data['Slide Number'] != 'PDF-6, Pg35']
data = data[data['Slide Number'] != 'PDF-6, Pg40']
data = data[data['Slide Number'] != 'PDF-6, Pg47']
data = data[data['Slide Number'] != 'PDF-6, Pg59']
data = data[data['Slide Number'] != 'PDF-6, Pg77']
data = data[data['Slide Number'] != 'PDF-6, Pg89']
data = data[data['Slide Number'] != 'PDF-6, Pg103']

data = data[data['Slide Number'] != 'PDF-8, Pg1']
data = data[data['Slide Number'] != 'PDF-8, Pg14']
data = data[data['Slide Number'] != 'PDF-8, Pg18']
data = data[data['Slide Number'] != 'PDF-8, Pg27']
data = data[data['Slide Number'] != 'PDF-8, Pg35']
data = data[data['Slide Number'] != 'PDF-8, Pg41']
data = data[data['Slide Number'] != 'PDF-8, Pg49']
data = data[data['Slide Number'] != 'PDF-8, Pg57']
data = data[data['Slide Number'] != 'PDF-8, Pg60']
data = data[data['Slide Number'] != 'PDF-8, Pg65']

data['Content'] = data['Content'].apply(lambda x: process_slides(x))

In [10]:
def get_bert_encodings(x):
    model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
    embeddings = model.encode(x)
    return embeddings

data['encodings'] = data['Content'].apply(lambda x: get_bert_encodings(x))

In [11]:
def get_query(tid, error_class):
    query1 = get_task_description(tid) 
    query2 = get_error_description(error_class)
    query = query1 + ' ' + query2
    return(query)


In [12]:
tqdm.pandas(desc="Processing...")
df['Enriched_Query'] = df.apply(lambda x: get_query(x['taskid'], x['Severe Error']), axis=1)
df['Bert_Encodings'] = df['Enriched_Query'].progress_apply(lambda x: get_bert_encodings(x))
df             

Processing...: 100%|████████████████████████| 4323/4323 [03:50<00:00, 18.74it/s]


,index,ID,UserID,UserQuery,errorClasses,taskid,timestamp,E0,E1,E2,...,E15,E16,E17,E18,E19,E20,E21,Severe Error,Enriched_Query,Bert_Encodings
0,29,26290,1143,SELECT Name FROM Customer;,0,454,2020-06-15 12:07:01,1,0,0,...,0,0,0,0,0,0,0,E0,select the names of all custumers. Get the sy...,"[0.045353487, 0.026892075, -0.08102727, -0.021..."
1,44,33600,1158,"SELECT customer.Cid, customer.NameFROM Custome...",0,454,2020-06-26 08:43:20,1,0,0,...,0,0,0,0,0,0,0,E0,select the names of all custumers. Get the sy...,"[0.045353487, 0.026892075, -0.08102727, -0.021..."
2,70,28417,1216,select * from Customer;,2,454,2020-06-18 18:17:16,0,0,1,...,0,0,0,0,0,0,0,E2,select the names of all custumers. Give examp...,"[0.018104844, 0.03776139, -0.016927559, -0.016..."
3,87,28121,1246,"select cid, name FROM customer GROUP BY name",2,454,2020-06-18 12:01:15,0,0,1,...,0,0,0,0,0,0,0,E2,select the names of all custumers. Give examp...,"[0.018104844, 0.03776139, -0.016927559, -0.016..."
4,98,31056,1277,select * from customer,2,454,2020-06-20 20:42:07,0,0,1,...,0,0,0,0,0,0,0,E2,select the names of all custumers. Give examp...,"[0.018104844, 0.03776139, -0.016927559, -0.016..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,50164,199669,2475,CREATE TABLE WINE( name VARCHAR (50) DEFAULT ...,0,1219,2023-07-26 14:43:12,1,0,0,...,0,0,0,0,0,0,0,E0,"Given the example database in the appendix, cr...","[0.10937697, 0.014218844, -0.09684275, 0.06378..."
4319,50179,199527,2479,CREATE TABLE WEIN(farbe varchar(100) DEFAULT (...,0,1219,2023-07-25 19:40:44,1,0,0,...,0,0,0,0,0,0,0,E0,"Given the example database in the appendix, cr...","[0.10937697, 0.014218844, -0.09684275, 0.06378..."
4320,50184,199691,2479,CREATE TABLE WEIN( farbe varchar(100) DEFAULT...,0,1219,2023-07-26 15:29:23,1,0,0,...,0,0,0,0,0,0,0,E0,"Given the example database in the appendix, cr...","[0.10937697, 0.014218844, -0.09684275, 0.06378..."
4321,50211,190044,2278,create table PRODUCER( vineyard varchar(20) n...,0,1220,2023-05-04 14:05:29,1,0,0,...,0,0,0,0,0,0,0,E0,create the tables for the table PRODUCER. Get...,"[0.085943624, -0.05354213, -0.08145538, 0.0530..."


In [13]:
def rec(keyword_encodings):
    recc = ''
    data['similarity_score'] = data['encodings'].apply(lambda x: util.dot_score(x, keyword_encodings))
    df_results = data.sort_values(by=['similarity_score'], ascending=False)
    df_results = df_results.reset_index()
    recc = df_results['Slide Number'][0] + ';' + df_results['Slide Number'][1] + ';' + df_results['Slide Number'][2]
    return recc


df['Recommendations'] = df['Bert_Encodings'].progress_apply(lambda x: rec(x))
df
    

Processing...: 100%|███████████████████████| 4323/4323 [00:15<00:00, 278.32it/s]


,index,ID,UserID,UserQuery,errorClasses,taskid,timestamp,E0,E1,E2,...,E16,E17,E18,E19,E20,E21,Severe Error,Enriched_Query,Bert_Encodings,Recommendations
0,29,26290,1143,SELECT Name FROM Customer;,0,454,2020-06-15 12:07:01,1,0,0,...,0,0,0,0,0,0,E0,select the names of all custumers. Get the sy...,"[0.045353487, 0.026892075, -0.08102727, -0.021...","PDF-6, Pg52;PDF-6, Pg68;PDF-6, Pg15"
1,44,33600,1158,"SELECT customer.Cid, customer.NameFROM Custome...",0,454,2020-06-26 08:43:20,1,0,0,...,0,0,0,0,0,0,E0,select the names of all custumers. Get the sy...,"[0.045353487, 0.026892075, -0.08102727, -0.021...","PDF-6, Pg52;PDF-6, Pg68;PDF-6, Pg15"
2,70,28417,1216,select * from Customer;,2,454,2020-06-18 18:17:16,0,0,1,...,0,0,0,0,0,0,E2,select the names of all custumers. Give examp...,"[0.018104844, 0.03776139, -0.016927559, -0.016...","PDF-6, Pg23;PDF-6, Pg15;PDF-6, Pg27"
3,87,28121,1246,"select cid, name FROM customer GROUP BY name",2,454,2020-06-18 12:01:15,0,0,1,...,0,0,0,0,0,0,E2,select the names of all custumers. Give examp...,"[0.018104844, 0.03776139, -0.016927559, -0.016...","PDF-6, Pg23;PDF-6, Pg15;PDF-6, Pg27"
4,98,31056,1277,select * from customer,2,454,2020-06-20 20:42:07,0,0,1,...,0,0,0,0,0,0,E2,select the names of all custumers. Give examp...,"[0.018104844, 0.03776139, -0.016927559, -0.016...","PDF-6, Pg23;PDF-6, Pg15;PDF-6, Pg27"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,50164,199669,2475,CREATE TABLE WINE( name VARCHAR (50) DEFAULT ...,0,1219,2023-07-26 14:43:12,1,0,0,...,0,0,0,0,0,0,E0,"Given the example database in the appendix, cr...","[0.10937697, 0.014218844, -0.09684275, 0.06378...","PDF-2, Pg26;PDF-2, Pg45;PDF-2, Pg20"
4319,50179,199527,2479,CREATE TABLE WEIN(farbe varchar(100) DEFAULT (...,0,1219,2023-07-25 19:40:44,1,0,0,...,0,0,0,0,0,0,E0,"Given the example database in the appendix, cr...","[0.10937697, 0.014218844, -0.09684275, 0.06378...","PDF-2, Pg26;PDF-2, Pg45;PDF-2, Pg20"
4320,50184,199691,2479,CREATE TABLE WEIN( farbe varchar(100) DEFAULT...,0,1219,2023-07-26 15:29:23,1,0,0,...,0,0,0,0,0,0,E0,"Given the example database in the appendix, cr...","[0.10937697, 0.014218844, -0.09684275, 0.06378...","PDF-2, Pg26;PDF-2, Pg45;PDF-2, Pg20"
4321,50211,190044,2278,create table PRODUCER( vineyard varchar(20) n...,0,1220,2023-05-04 14:05:29,1,0,0,...,0,0,0,0,0,0,E0,create the tables for the table PRODUCER. Get...,"[0.085943624, -0.05354213, -0.08145538, 0.0530...","PDF-6, Pg15;PDF-2, Pg19;PDF-2, Pg17"


In [21]:
def give_instructions(error_class, recommendation):
    action = "Learn"
    instruction_synonyms = {
    "Learn": ["Learn", "Study", "Review", "Access", "Read", "Go to", "Navigate to"]
    }
    instruction_synonym = random.choice(instruction_synonyms.get(action, [""]))
    
    # Generate instruction
    
    if error_class == "E0":
      instruction_text = f"{instruction_synonym} {recommendation} covering notation."
    elif error_class == "E2":
      instruction_text = f"{instruction_synonym} {recommendation} discussing count errors."
    elif error_class == "E4":
      instruction_text = f"{instruction_synonym} {recommendation} addressing table name errors."
    elif error_class == "E5":
      instruction_text = f"{instruction_synonym} {recommendation} addressing column count errors in CT."
    elif error_class == "E6":
      instruction_text = f"{instruction_synonym} {recommendation} addressing table name errors."
    elif error_class == "E8":
      instruction_text = f"{instruction_synonym} {recommendation} addressing table name errors."
    elif error_class == "E11":
      instruction_text = f"{instruction_synonym} {recommendation} addressing table name errors."
    elif error_class == "E12":
      instruction_text = f"{instruction_synonym} {recommendation} addressing table name errors."
    elif error_class in ['E14', 'E15']:
      instruction_text = f"{instruction_synonym} {recommendation} addressing table name errors."
    elif error_class == "E16":
      instruction_text = f"{instruction_synonym} {recommendation} addressing table name errors."
    elif error_class == "E20":
      instruction_text = f"{instruction_synonym} {recommendation} addressing table content errors."

    return(instruction_text)

df['Suggested Action'] = df.apply(lambda x: give_instructions(x['Severe Error'], x['Recommendations']), axis=1)
df

,index,ID,UserID,UserQuery,errorClasses,taskid,timestamp,E0,E1,E2,...,E17,E18,E19,E20,E21,Severe Error,Enriched_Query,Bert_Encodings,Recommendations,Suggested Action
0,29,26290,1143,SELECT Name FROM Customer;,0,454,2020-06-15 12:07:01,1,0,0,...,0,0,0,0,0,E0,select the names of all custumers. Get the sy...,"[0.045353487, 0.026892075, -0.08102727, -0.021...","PDF-6, Pg52;PDF-6, Pg68;PDF-6, Pg15","Go to PDF-6, Pg52;PDF-6, Pg68;PDF-6, Pg15 cove..."
1,44,33600,1158,"SELECT customer.Cid, customer.NameFROM Custome...",0,454,2020-06-26 08:43:20,1,0,0,...,0,0,0,0,0,E0,select the names of all custumers. Get the sy...,"[0.045353487, 0.026892075, -0.08102727, -0.021...","PDF-6, Pg52;PDF-6, Pg68;PDF-6, Pg15","Study PDF-6, Pg52;PDF-6, Pg68;PDF-6, Pg15 cove..."
2,70,28417,1216,select * from Customer;,2,454,2020-06-18 18:17:16,0,0,1,...,0,0,0,0,0,E2,select the names of all custumers. Give examp...,"[0.018104844, 0.03776139, -0.016927559, -0.016...","PDF-6, Pg23;PDF-6, Pg15;PDF-6, Pg27","Review PDF-6, Pg23;PDF-6, Pg15;PDF-6, Pg27 dis..."
3,87,28121,1246,"select cid, name FROM customer GROUP BY name",2,454,2020-06-18 12:01:15,0,0,1,...,0,0,0,0,0,E2,select the names of all custumers. Give examp...,"[0.018104844, 0.03776139, -0.016927559, -0.016...","PDF-6, Pg23;PDF-6, Pg15;PDF-6, Pg27","Review PDF-6, Pg23;PDF-6, Pg15;PDF-6, Pg27 dis..."
4,98,31056,1277,select * from customer,2,454,2020-06-20 20:42:07,0,0,1,...,0,0,0,0,0,E2,select the names of all custumers. Give examp...,"[0.018104844, 0.03776139, -0.016927559, -0.016...","PDF-6, Pg23;PDF-6, Pg15;PDF-6, Pg27","Go to PDF-6, Pg23;PDF-6, Pg15;PDF-6, Pg27 disc..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,50164,199669,2475,CREATE TABLE WINE( name VARCHAR (50) DEFAULT ...,0,1219,2023-07-26 14:43:12,1,0,0,...,0,0,0,0,0,E0,"Given the example database in the appendix, cr...","[0.10937697, 0.014218844, -0.09684275, 0.06378...","PDF-2, Pg26;PDF-2, Pg45;PDF-2, Pg20","Study PDF-2, Pg26;PDF-2, Pg45;PDF-2, Pg20 cove..."
4319,50179,199527,2479,CREATE TABLE WEIN(farbe varchar(100) DEFAULT (...,0,1219,2023-07-25 19:40:44,1,0,0,...,0,0,0,0,0,E0,"Given the example database in the appendix, cr...","[0.10937697, 0.014218844, -0.09684275, 0.06378...","PDF-2, Pg26;PDF-2, Pg45;PDF-2, Pg20","Learn PDF-2, Pg26;PDF-2, Pg45;PDF-2, Pg20 cove..."
4320,50184,199691,2479,CREATE TABLE WEIN( farbe varchar(100) DEFAULT...,0,1219,2023-07-26 15:29:23,1,0,0,...,0,0,0,0,0,E0,"Given the example database in the appendix, cr...","[0.10937697, 0.014218844, -0.09684275, 0.06378...","PDF-2, Pg26;PDF-2, Pg45;PDF-2, Pg20","Access PDF-2, Pg26;PDF-2, Pg45;PDF-2, Pg20 cov..."
4321,50211,190044,2278,create table PRODUCER( vineyard varchar(20) n...,0,1220,2023-05-04 14:05:29,1,0,0,...,0,0,0,0,0,E0,create the tables for the table PRODUCER. Get...,"[0.085943624, -0.05354213, -0.08145538, 0.0530...","PDF-6, Pg15;PDF-2, Pg19;PDF-2, Pg17","Go to PDF-6, Pg15;PDF-2, Pg19;PDF-2, Pg17 cove..."


In [22]:
dataFile = pd.read_excel('DBC20-23.xlsx',sheet_name='Ground_truth')
dataFile

,TaskID,ErrorClass,Slide Number
0,456,0,"PDF-6, Pg58;PDF-6, Pg54;PDF-2, Pg39"
1,456,2,"PDF-6, Pg58;PDF-6, Pg37;PDF-2, Pg39"
2,456,6,"PDF-2, Pg39;PDF-6, Pg58;PDF-6, Pg54"
3,459,0,"PDF-6, Pg18;PDF-6, Pg21;PDF-2, Pg40"
4,459,2,"PDF-6, Pg21;PDF-2, Pg40"
5,464,0,"PDF-6, Pg23;PDF-6, Pg32"
6,464,2,"PDF-6, Pg32;PDF-6, Pg23;PDF-6, Pg14"
7,465,0,"PDF-6, Pg30;PDF-6, Pg32;PDF-6, Pg42"
8,465,2,"PDF-6, Pg32; PDF-6, Pg30"
9,465,6,"PDF-6, Pg32; PDF-6, Pg30"


In [23]:
df = df.drop_duplicates(subset=['taskid', 'Severe Error'])
df2 = df[:40]
df2.reset_index()

,level_0,index,ID,UserID,UserQuery,errorClasses,taskid,timestamp,E0,E1,...,E17,E18,E19,E20,E21,Severe Error,Enriched_Query,Bert_Encodings,Recommendations,Suggested Action
0,0,29,26290,1143,SELECT Name FROM Customer;,0,454,2020-06-15 12:07:01,1,0,...,0,0,0,0,0,E0,select the names of all custumers. Get the sy...,"[0.045353487, 0.026892075, -0.08102727, -0.021...","PDF-6, Pg52;PDF-6, Pg68;PDF-6, Pg15","Go to PDF-6, Pg52;PDF-6, Pg68;PDF-6, Pg15 cove..."
1,2,70,28417,1216,select * from Customer;,2,454,2020-06-18 18:17:16,0,0,...,0,0,0,0,0,E2,select the names of all custumers. Give examp...,"[0.018104844, 0.03776139, -0.016927559, -0.016...","PDF-6, Pg23;PDF-6, Pg15;PDF-6, Pg27","Review PDF-6, Pg23;PDF-6, Pg15;PDF-6, Pg27 dis..."
2,11,145,26380,1101,"select O.Oid, O.Did, O.Date, O.Cidfrom Orders ...",0,455,2020-06-15 13:20:52,1,0,...,0,0,0,0,0,E0,select all orders of customer Meier. Get the ...,"[0.026299462, 0.03945725, -0.017864216, -0.037...","PDF-6, Pg82;PDF-6, Pg34;PDF-6, Pg15","Access PDF-6, Pg82;PDF-6, Pg34;PDF-6, Pg15 cov..."
3,13,157,27291,1103,"SELECT Customer.name, Orders.OidFROM CustomerJ...",2,455,2020-06-16 23:33:31,0,0,...,0,0,0,0,0,E2,select all orders of customer Meier. Give exa...,"[-0.00024898243, 0.05706003, 0.037423924, -0.0...","PDF-6, Pg82;PDF-6, Pg34;PDF-6, Pg54","Go to PDF-6, Pg82;PDF-6, Pg34;PDF-6, Pg54 disc..."
4,21,201,26352,1111,"select oid, did, date, customer.cid from cust...",6,455,2020-06-15 12:53:25,0,0,...,0,0,0,0,0,E6,select all orders of customer Meier. Adjust t...,"[0.054789763, 0.0054591103, 0.037330564, -0.03...","PDF-8, Pg32;PDF-6, Pg70;PDF-2, Pg26","Read PDF-8, Pg32;PDF-6, Pg70;PDF-2, Pg26 addre..."
5,95,816,26649,1101,"select P.Pid, P.labelfrom Product as Pinner jo...",0,456,2020-06-15 17:00:58,1,0,...,0,0,0,0,0,E0,select all products where have not been sold o...,"[0.008381918, 0.0007661434, -0.014332468, 0.01...","PDF-6, Pg58;PDF-6, Pg54;PDF-6, Pg68","Go to PDF-6, Pg58;PDF-6, Pg54;PDF-6, Pg68 cove..."
6,102,861,30280,1103,select * from product left join Line_item on p...,2,456,2020-06-20 17:09:46,0,0,...,0,0,0,0,0,E2,select all products where have not been sold o...,"[-0.005690071, 0.009727106, 0.020480538, 0.003...","PDF-6, Pg58;PDF-6, Pg54;PDF-6, Pg37","Study PDF-6, Pg58;PDF-6, Pg54;PDF-6, Pg37 disc..."
7,128,1028,27131,1113,"select p.Pid, p.Labelfrom Orders o, Line_Item ...",6,456,2020-06-16 15:43:53,0,0,...,0,0,0,0,0,E6,select all products where have not been sold o...,"[0.036138017, -0.02662367, 0.015946142, -0.013...","PDF-2, Pg26;PDF-6, Pg54;PDF-6, Pg58","Learn PDF-2, Pg26;PDF-6, Pg54;PDF-6, Pg58 addr..."
8,222,1875,27657,1101,"select P.Pid, P.Labelfrom Product as Pinner jo...",6,457,2020-06-17 15:42:33,0,0,...,0,0,0,0,0,E6,select all products where dealer Schmidt sold ...,"[0.058459684, 0.009413479, -0.05943552, -0.013...","PDF-2, Pg26;PDF-6, Pg37;PDF-6, Pg21","Read PDF-2, Pg26;PDF-6, Pg37;PDF-6, Pg21 addre..."
9,223,1888,30124,1103,select* from Product LEFT JOIN Line_item on P...,2,457,2020-06-20 15:18:58,0,0,...,0,0,0,0,0,E2,select all products where dealer Schmidt sold ...,"[0.028462024, 0.03517126, -0.054934632, -0.014...","PDF-6, Pg16;PDF-6, Pg21;PDF-6, Pg24","Access PDF-6, Pg16;PDF-6, Pg21;PDF-6, Pg24 dis..."


In [24]:
#Evaluation

actual = []
taskid = list(df2['taskid'])
error_class = list(df2['Severe Error'])
recommendation = list(df2['Recommendations'])        

for i in range(len(taskid)):
    task_id_val = int(taskid[i])
    error_class_val = int(error_class[i][1:])
    s = dataFile.loc[(dataFile['TaskID'] == task_id_val) & (dataFile['ErrorClass'] == error_class_val), 'Slide Number']
    s = str(s)
    pattern = r'PDF-\d+, Pg\d+'
    matches = re.findall(pattern, s)
    result = ';'.join(matches)
    actual.append(result)
    
eval_data = pd.DataFrame({'TaskID': taskid, 'Error_Class': error_class, 'Recommendation':recommendation, 'Actual':actual})
eval_data.to_csv('eval_data.csv')
eval_data

,TaskID,Error_Class,Recommendation,Actual
0,454,E0,"PDF-6, Pg52;PDF-6, Pg68;PDF-6, Pg15","PDF-6, Pg15;PDF-6, Pg68;PDF-6, Pg27"
1,454,E2,"PDF-6, Pg23;PDF-6, Pg15;PDF-6, Pg27","PDF-6, Pg23;PDF-6, Pg27"
2,455,E0,"PDF-6, Pg82;PDF-6, Pg34;PDF-6, Pg15","PDF-6, Pg34;PDF-6, Pg33"
3,455,E2,"PDF-6, Pg82;PDF-6, Pg34;PDF-6, Pg54","PDF-6, Pg34;PDF-6, Pg54;PDF-6, Pg33"
4,455,E6,"PDF-8, Pg32;PDF-6, Pg70;PDF-2, Pg26","PDF-2, Pg26;PDF-6, Pg34"
5,456,E0,"PDF-6, Pg58;PDF-6, Pg54;PDF-6, Pg68","PDF-6, Pg58;PDF-6, Pg54;PDF-2, Pg39"
6,456,E2,"PDF-6, Pg58;PDF-6, Pg54;PDF-6, Pg37","PDF-6, Pg58;PDF-6, Pg37;PDF-2, Pg39"
7,456,E6,"PDF-2, Pg26;PDF-6, Pg54;PDF-6, Pg58","PDF-2, Pg39;PDF-6, Pg58;PDF-6, Pg54"
8,457,E6,"PDF-2, Pg26;PDF-6, Pg37;PDF-6, Pg21","PDF-6, Pg21;PDF-6, Pg42;PDF-6, Pg20"
9,457,E2,"PDF-6, Pg16;PDF-6, Pg21;PDF-6, Pg24","PDF-6, Pg21;PDF-6, Pg42;PDF-6, Pg20"


In [25]:
eval_data['top1_reccom']=eval_data['Recommendation'].apply(lambda x:[x.split(';')[0]])
eval_data['top2_reccom']=eval_data['Recommendation'].apply(lambda x:x.split(';')[:2])
eval_data['top3_reccom']=eval_data['Recommendation'].apply(lambda x:x.split(';')[:3])

eval_data

,TaskID,Error_Class,Recommendation,Actual,top1_reccom,top2_reccom,top3_reccom
0,454,E0,"PDF-6, Pg52;PDF-6, Pg68;PDF-6, Pg15","PDF-6, Pg15;PDF-6, Pg68;PDF-6, Pg27","[PDF-6, Pg52]","[PDF-6, Pg52, PDF-6, Pg68]","[PDF-6, Pg52, PDF-6, Pg68, PDF-6, Pg15]"
1,454,E2,"PDF-6, Pg23;PDF-6, Pg15;PDF-6, Pg27","PDF-6, Pg23;PDF-6, Pg27","[PDF-6, Pg23]","[PDF-6, Pg23, PDF-6, Pg15]","[PDF-6, Pg23, PDF-6, Pg15, PDF-6, Pg27]"
2,455,E0,"PDF-6, Pg82;PDF-6, Pg34;PDF-6, Pg15","PDF-6, Pg34;PDF-6, Pg33","[PDF-6, Pg82]","[PDF-6, Pg82, PDF-6, Pg34]","[PDF-6, Pg82, PDF-6, Pg34, PDF-6, Pg15]"
3,455,E2,"PDF-6, Pg82;PDF-6, Pg34;PDF-6, Pg54","PDF-6, Pg34;PDF-6, Pg54;PDF-6, Pg33","[PDF-6, Pg82]","[PDF-6, Pg82, PDF-6, Pg34]","[PDF-6, Pg82, PDF-6, Pg34, PDF-6, Pg54]"
4,455,E6,"PDF-8, Pg32;PDF-6, Pg70;PDF-2, Pg26","PDF-2, Pg26;PDF-6, Pg34","[PDF-8, Pg32]","[PDF-8, Pg32, PDF-6, Pg70]","[PDF-8, Pg32, PDF-6, Pg70, PDF-2, Pg26]"
5,456,E0,"PDF-6, Pg58;PDF-6, Pg54;PDF-6, Pg68","PDF-6, Pg58;PDF-6, Pg54;PDF-2, Pg39","[PDF-6, Pg58]","[PDF-6, Pg58, PDF-6, Pg54]","[PDF-6, Pg58, PDF-6, Pg54, PDF-6, Pg68]"
6,456,E2,"PDF-6, Pg58;PDF-6, Pg54;PDF-6, Pg37","PDF-6, Pg58;PDF-6, Pg37;PDF-2, Pg39","[PDF-6, Pg58]","[PDF-6, Pg58, PDF-6, Pg54]","[PDF-6, Pg58, PDF-6, Pg54, PDF-6, Pg37]"
7,456,E6,"PDF-2, Pg26;PDF-6, Pg54;PDF-6, Pg58","PDF-2, Pg39;PDF-6, Pg58;PDF-6, Pg54","[PDF-2, Pg26]","[PDF-2, Pg26, PDF-6, Pg54]","[PDF-2, Pg26, PDF-6, Pg54, PDF-6, Pg58]"
8,457,E6,"PDF-2, Pg26;PDF-6, Pg37;PDF-6, Pg21","PDF-6, Pg21;PDF-6, Pg42;PDF-6, Pg20","[PDF-2, Pg26]","[PDF-2, Pg26, PDF-6, Pg37]","[PDF-2, Pg26, PDF-6, Pg37, PDF-6, Pg21]"
9,457,E2,"PDF-6, Pg16;PDF-6, Pg21;PDF-6, Pg24","PDF-6, Pg21;PDF-6, Pg42;PDF-6, Pg20","[PDF-6, Pg16]","[PDF-6, Pg16, PDF-6, Pg21]","[PDF-6, Pg16, PDF-6, Pg21, PDF-6, Pg24]"


In [34]:
for index,row in eval_data.iterrows():
    actual=row['Actual'].split(';')
    predcited1=row['top1_reccom']
    predcited2=row['top2_reccom']
    predcited3=row['top3_reccom']
    
    tp1=len(set(actual)&set(predcited1))
    fp1=len(set(predcited1)-set(actual))
    fn1=len(set(actual)-set(predcited1))
    eval_data.loc[index,'tp_1']=tp1
    eval_data.loc[index,'fp_1']=fp1
    eval_data.loc[index,'fn_1']=fn1
    
    tp2=len(set(actual)&set(predcited2))
    fp2=len(set(predcited2)-set(actual))
    fn2=len(set(actual)-set(predcited2))
    eval_data.loc[index,'tp_2']=tp2
    eval_data.loc[index,'fp_2']=fp2
    eval_data.loc[index,'fn_2']=fn2
    
    tp3=len(set(actual)&set(predcited3))
    fp3=len(set(predcited3)-set(actual))
    fn3=len(set(actual)-set(predcited3))
    eval_data.loc[index,'tp_3']=tp3
    eval_data.loc[index,'fp_3']=fp3
    eval_data.loc[index,'fn_3']=fn3

In [35]:
eval_data['precision1']=eval_data['tp_1']/(eval_data['tp_1']+eval_data['fp_1'])
eval_data['recall1']=eval_data['tp_1']/(eval_data['tp_1']+eval_data['fn_1'])
eval_data

,TaskID,Error_Class,Recommendation,Actual,top1_reccom,top2_reccom,top3_reccom,tp_1,fp_1,fn_1,...,fn_2,tp_3,fp_3,fn_3,precision1,recall1,precision3,recall3,precision2,recall2
0,454,E0,"PDF-6, Pg52;PDF-6, Pg68;PDF-6, Pg15","PDF-6, Pg15;PDF-6, Pg68;PDF-6, Pg27","[PDF-6, Pg52]","[PDF-6, Pg52, PDF-6, Pg68]","[PDF-6, Pg52, PDF-6, Pg68, PDF-6, Pg15]",0.0,1.0,3.0,...,2.0,2.0,1.0,1.0,0.0,0.000000,0.666667,0.666667,0.5,0.333333
1,454,E2,"PDF-6, Pg23;PDF-6, Pg15;PDF-6, Pg27","PDF-6, Pg23;PDF-6, Pg27","[PDF-6, Pg23]","[PDF-6, Pg23, PDF-6, Pg15]","[PDF-6, Pg23, PDF-6, Pg15, PDF-6, Pg27]",1.0,0.0,1.0,...,1.0,2.0,1.0,0.0,1.0,0.500000,0.666667,1.000000,0.5,0.500000
2,455,E0,"PDF-6, Pg82;PDF-6, Pg34;PDF-6, Pg15","PDF-6, Pg34;PDF-6, Pg33","[PDF-6, Pg82]","[PDF-6, Pg82, PDF-6, Pg34]","[PDF-6, Pg82, PDF-6, Pg34, PDF-6, Pg15]",0.0,1.0,2.0,...,1.0,1.0,2.0,1.0,0.0,0.000000,0.333333,0.500000,0.5,0.500000
3,455,E2,"PDF-6, Pg82;PDF-6, Pg34;PDF-6, Pg54","PDF-6, Pg34;PDF-6, Pg54;PDF-6, Pg33","[PDF-6, Pg82]","[PDF-6, Pg82, PDF-6, Pg34]","[PDF-6, Pg82, PDF-6, Pg34, PDF-6, Pg54]",0.0,1.0,3.0,...,2.0,2.0,1.0,1.0,0.0,0.000000,0.666667,0.666667,0.5,0.333333
4,455,E6,"PDF-8, Pg32;PDF-6, Pg70;PDF-2, Pg26","PDF-2, Pg26;PDF-6, Pg34","[PDF-8, Pg32]","[PDF-8, Pg32, PDF-6, Pg70]","[PDF-8, Pg32, PDF-6, Pg70, PDF-2, Pg26]",0.0,1.0,2.0,...,2.0,1.0,2.0,1.0,0.0,0.000000,0.333333,0.500000,0.0,0.000000
5,456,E0,"PDF-6, Pg58;PDF-6, Pg54;PDF-6, Pg68","PDF-6, Pg58;PDF-6, Pg54;PDF-2, Pg39","[PDF-6, Pg58]","[PDF-6, Pg58, PDF-6, Pg54]","[PDF-6, Pg58, PDF-6, Pg54, PDF-6, Pg68]",1.0,0.0,2.0,...,1.0,2.0,1.0,1.0,1.0,0.333333,0.666667,0.666667,1.0,0.666667
6,456,E2,"PDF-6, Pg58;PDF-6, Pg54;PDF-6, Pg37","PDF-6, Pg58;PDF-6, Pg37;PDF-2, Pg39","[PDF-6, Pg58]","[PDF-6, Pg58, PDF-6, Pg54]","[PDF-6, Pg58, PDF-6, Pg54, PDF-6, Pg37]",1.0,0.0,2.0,...,2.0,2.0,1.0,1.0,1.0,0.333333,0.666667,0.666667,0.5,0.333333
7,456,E6,"PDF-2, Pg26;PDF-6, Pg54;PDF-6, Pg58","PDF-2, Pg39;PDF-6, Pg58;PDF-6, Pg54","[PDF-2, Pg26]","[PDF-2, Pg26, PDF-6, Pg54]","[PDF-2, Pg26, PDF-6, Pg54, PDF-6, Pg58]",0.0,1.0,3.0,...,2.0,2.0,1.0,1.0,0.0,0.000000,0.666667,0.666667,0.5,0.333333
8,457,E6,"PDF-2, Pg26;PDF-6, Pg37;PDF-6, Pg21","PDF-6, Pg21;PDF-6, Pg42;PDF-6, Pg20","[PDF-2, Pg26]","[PDF-2, Pg26, PDF-6, Pg37]","[PDF-2, Pg26, PDF-6, Pg37, PDF-6, Pg21]",0.0,1.0,3.0,...,3.0,1.0,2.0,2.0,0.0,0.000000,0.333333,0.333333,0.0,0.000000
9,457,E2,"PDF-6, Pg16;PDF-6, Pg21;PDF-6, Pg24","PDF-6, Pg21;PDF-6, Pg42;PDF-6, Pg20","[PDF-6, Pg16]","[PDF-6, Pg16, PDF-6, Pg21]","[PDF-6, Pg16, PDF-6, Pg21, PDF-6, Pg24]",0.0,1.0,3.0,...,2.0,1.0,2.0,2.0,0.0,0.000000,0.333333,0.333333,0.5,0.333333


In [28]:
eval_data['precision3']=eval_data['tp_3']/(eval_data['tp_3']+eval_data['fp_3'])
eval_data['recall3']=eval_data['tp_3']/(eval_data['tp_3']+eval_data['fn_3'])
eval_data

,TaskID,Error_Class,Recommendation,Actual,top1_reccom,top2_reccom,top3_reccom,tp_1,fp_1,fn_1,tp_2,fp_2,fn_2,tp_3,fp_3,fn_3,precision1,recall1,precision3,recall3
0,454,E0,"PDF-6, Pg52;PDF-6, Pg68;PDF-6, Pg15","PDF-6, Pg15;PDF-6, Pg68;PDF-6, Pg27","[PDF-6, Pg52]","[PDF-6, Pg52, PDF-6, Pg68]","[PDF-6, Pg52, PDF-6, Pg68, PDF-6, Pg15]",0.0,1.0,3.0,1.0,1.0,2.0,2.0,1.0,1.0,0.0,0.000000,0.666667,0.666667
1,454,E2,"PDF-6, Pg23;PDF-6, Pg15;PDF-6, Pg27","PDF-6, Pg23;PDF-6, Pg27","[PDF-6, Pg23]","[PDF-6, Pg23, PDF-6, Pg15]","[PDF-6, Pg23, PDF-6, Pg15, PDF-6, Pg27]",1.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0,1.0,0.500000,0.666667,1.000000
2,455,E0,"PDF-6, Pg82;PDF-6, Pg34;PDF-6, Pg15","PDF-6, Pg34;PDF-6, Pg33","[PDF-6, Pg82]","[PDF-6, Pg82, PDF-6, Pg34]","[PDF-6, Pg82, PDF-6, Pg34, PDF-6, Pg15]",0.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0,0.000000,0.333333,0.500000
3,455,E2,"PDF-6, Pg82;PDF-6, Pg34;PDF-6, Pg54","PDF-6, Pg34;PDF-6, Pg54;PDF-6, Pg33","[PDF-6, Pg82]","[PDF-6, Pg82, PDF-6, Pg34]","[PDF-6, Pg82, PDF-6, Pg34, PDF-6, Pg54]",0.0,1.0,3.0,1.0,1.0,2.0,2.0,1.0,1.0,0.0,0.000000,0.666667,0.666667
4,455,E6,"PDF-8, Pg32;PDF-6, Pg70;PDF-2, Pg26","PDF-2, Pg26;PDF-6, Pg34","[PDF-8, Pg32]","[PDF-8, Pg32, PDF-6, Pg70]","[PDF-8, Pg32, PDF-6, Pg70, PDF-2, Pg26]",0.0,1.0,2.0,0.0,2.0,2.0,1.0,2.0,1.0,0.0,0.000000,0.333333,0.500000
5,456,E0,"PDF-6, Pg58;PDF-6, Pg54;PDF-6, Pg68","PDF-6, Pg58;PDF-6, Pg54;PDF-2, Pg39","[PDF-6, Pg58]","[PDF-6, Pg58, PDF-6, Pg54]","[PDF-6, Pg58, PDF-6, Pg54, PDF-6, Pg68]",1.0,0.0,2.0,2.0,0.0,1.0,2.0,1.0,1.0,1.0,0.333333,0.666667,0.666667
6,456,E2,"PDF-6, Pg58;PDF-6, Pg54;PDF-6, Pg37","PDF-6, Pg58;PDF-6, Pg37;PDF-2, Pg39","[PDF-6, Pg58]","[PDF-6, Pg58, PDF-6, Pg54]","[PDF-6, Pg58, PDF-6, Pg54, PDF-6, Pg37]",1.0,0.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,0.333333,0.666667,0.666667
7,456,E6,"PDF-2, Pg26;PDF-6, Pg54;PDF-6, Pg58","PDF-2, Pg39;PDF-6, Pg58;PDF-6, Pg54","[PDF-2, Pg26]","[PDF-2, Pg26, PDF-6, Pg54]","[PDF-2, Pg26, PDF-6, Pg54, PDF-6, Pg58]",0.0,1.0,3.0,1.0,1.0,2.0,2.0,1.0,1.0,0.0,0.000000,0.666667,0.666667
8,457,E6,"PDF-2, Pg26;PDF-6, Pg37;PDF-6, Pg21","PDF-6, Pg21;PDF-6, Pg42;PDF-6, Pg20","[PDF-2, Pg26]","[PDF-2, Pg26, PDF-6, Pg37]","[PDF-2, Pg26, PDF-6, Pg37, PDF-6, Pg21]",0.0,1.0,3.0,0.0,2.0,3.0,1.0,2.0,2.0,0.0,0.000000,0.333333,0.333333
9,457,E2,"PDF-6, Pg16;PDF-6, Pg21;PDF-6, Pg24","PDF-6, Pg21;PDF-6, Pg42;PDF-6, Pg20","[PDF-6, Pg16]","[PDF-6, Pg16, PDF-6, Pg21]","[PDF-6, Pg16, PDF-6, Pg21, PDF-6, Pg24]",0.0,1.0,3.0,1.0,1.0,2.0,1.0,2.0,2.0,0.0,0.000000,0.333333,0.333333


In [29]:
eval_data['precision2']=eval_data['tp_2']/(eval_data['tp_2']+eval_data['fp_2'])
eval_data['recall2']=eval_data['tp_2']/(eval_data['tp_2']+eval_data['fn_2'])
eval_data

,TaskID,Error_Class,Recommendation,Actual,top1_reccom,top2_reccom,top3_reccom,tp_1,fp_1,fn_1,...,fn_2,tp_3,fp_3,fn_3,precision1,recall1,precision3,recall3,precision2,recall2
0,454,E0,"PDF-6, Pg52;PDF-6, Pg68;PDF-6, Pg15","PDF-6, Pg15;PDF-6, Pg68;PDF-6, Pg27","[PDF-6, Pg52]","[PDF-6, Pg52, PDF-6, Pg68]","[PDF-6, Pg52, PDF-6, Pg68, PDF-6, Pg15]",0.0,1.0,3.0,...,2.0,2.0,1.0,1.0,0.0,0.000000,0.666667,0.666667,0.5,0.333333
1,454,E2,"PDF-6, Pg23;PDF-6, Pg15;PDF-6, Pg27","PDF-6, Pg23;PDF-6, Pg27","[PDF-6, Pg23]","[PDF-6, Pg23, PDF-6, Pg15]","[PDF-6, Pg23, PDF-6, Pg15, PDF-6, Pg27]",1.0,0.0,1.0,...,1.0,2.0,1.0,0.0,1.0,0.500000,0.666667,1.000000,0.5,0.500000
2,455,E0,"PDF-6, Pg82;PDF-6, Pg34;PDF-6, Pg15","PDF-6, Pg34;PDF-6, Pg33","[PDF-6, Pg82]","[PDF-6, Pg82, PDF-6, Pg34]","[PDF-6, Pg82, PDF-6, Pg34, PDF-6, Pg15]",0.0,1.0,2.0,...,1.0,1.0,2.0,1.0,0.0,0.000000,0.333333,0.500000,0.5,0.500000
3,455,E2,"PDF-6, Pg82;PDF-6, Pg34;PDF-6, Pg54","PDF-6, Pg34;PDF-6, Pg54;PDF-6, Pg33","[PDF-6, Pg82]","[PDF-6, Pg82, PDF-6, Pg34]","[PDF-6, Pg82, PDF-6, Pg34, PDF-6, Pg54]",0.0,1.0,3.0,...,2.0,2.0,1.0,1.0,0.0,0.000000,0.666667,0.666667,0.5,0.333333
4,455,E6,"PDF-8, Pg32;PDF-6, Pg70;PDF-2, Pg26","PDF-2, Pg26;PDF-6, Pg34","[PDF-8, Pg32]","[PDF-8, Pg32, PDF-6, Pg70]","[PDF-8, Pg32, PDF-6, Pg70, PDF-2, Pg26]",0.0,1.0,2.0,...,2.0,1.0,2.0,1.0,0.0,0.000000,0.333333,0.500000,0.0,0.000000
5,456,E0,"PDF-6, Pg58;PDF-6, Pg54;PDF-6, Pg68","PDF-6, Pg58;PDF-6, Pg54;PDF-2, Pg39","[PDF-6, Pg58]","[PDF-6, Pg58, PDF-6, Pg54]","[PDF-6, Pg58, PDF-6, Pg54, PDF-6, Pg68]",1.0,0.0,2.0,...,1.0,2.0,1.0,1.0,1.0,0.333333,0.666667,0.666667,1.0,0.666667
6,456,E2,"PDF-6, Pg58;PDF-6, Pg54;PDF-6, Pg37","PDF-6, Pg58;PDF-6, Pg37;PDF-2, Pg39","[PDF-6, Pg58]","[PDF-6, Pg58, PDF-6, Pg54]","[PDF-6, Pg58, PDF-6, Pg54, PDF-6, Pg37]",1.0,0.0,2.0,...,2.0,2.0,1.0,1.0,1.0,0.333333,0.666667,0.666667,0.5,0.333333
7,456,E6,"PDF-2, Pg26;PDF-6, Pg54;PDF-6, Pg58","PDF-2, Pg39;PDF-6, Pg58;PDF-6, Pg54","[PDF-2, Pg26]","[PDF-2, Pg26, PDF-6, Pg54]","[PDF-2, Pg26, PDF-6, Pg54, PDF-6, Pg58]",0.0,1.0,3.0,...,2.0,2.0,1.0,1.0,0.0,0.000000,0.666667,0.666667,0.5,0.333333
8,457,E6,"PDF-2, Pg26;PDF-6, Pg37;PDF-6, Pg21","PDF-6, Pg21;PDF-6, Pg42;PDF-6, Pg20","[PDF-2, Pg26]","[PDF-2, Pg26, PDF-6, Pg37]","[PDF-2, Pg26, PDF-6, Pg37, PDF-6, Pg21]",0.0,1.0,3.0,...,3.0,1.0,2.0,2.0,0.0,0.000000,0.333333,0.333333,0.0,0.000000
9,457,E2,"PDF-6, Pg16;PDF-6, Pg21;PDF-6, Pg24","PDF-6, Pg21;PDF-6, Pg42;PDF-6, Pg20","[PDF-6, Pg16]","[PDF-6, Pg16, PDF-6, Pg21]","[PDF-6, Pg16, PDF-6, Pg21, PDF-6, Pg24]",0.0,1.0,3.0,...,2.0,1.0,2.0,2.0,0.0,0.000000,0.333333,0.333333,0.5,0.333333


In [38]:
#Precision
pr1 = eval_data['precision1'].mean()
pr2 = eval_data['precision2'].mean()
pr3 = eval_data['precision3'].mean()

print("Precision@1 = {}, Precision@2 = {}, Precision@3 = {}" .format(pr1,pr2,pr3))

Precision@1 = 0.45, Precision@2 = 0.5625, Precision@3 = 0.5583333333333332


In [39]:
#Recall
r1 = eval_data['recall1'].mean()
r2 = eval_data['recall2'].mean()
r3 = eval_data['recall3'].mean()

print("Recall@1 = {}, Recall@2 = {}, Recall@3 = {}" .format(r1,r2,r3))

Recall@1 = 0.16666666666666666, Recall@2 = 0.4125, Recall@3 = 0.625


In [40]:
#F1 Score
f1 = (2*pr3*r3)/(pr3+r3)
f1

0.5897887323943661